# 📘 data_preparation.ipynb (Step-by-Step Cells for Dataset Engineering)

### Step 1: Load the CSV and initialize new columns


In [ ]:
import pandas as pd

# Filepath to the CSV file
file_path = "/Users/chrisloukasntais/VsCode/EchoCanvas/dataset.csv"

# Load the CSV file
df = pd.read_csv(file_path)

# Drop duplicates based on the "author_name" column
df_unique = df.drop_duplicates(subset=["author_name"])

# Randomly sample 1000 rows from the unique dataset
df_sampled = df_unique.sample(n=1000, random_state=42)

# Save the sampled data to a new CSV file
output_file_path = "/Users/chrisloukasntais/VsCode/EchoCanvas/dataset.csv"
df_sampled.to_csv(output_file_path, index=False)

print(f"Random 1000 unique samples saved to: {output_file_path}")

/var/folders/bc/8_g2sm9x4917kl9d15xv74cc0000gn/T/ipykernel_1788/3411192586.py:7: DtypeWarning: Columns (3,5,6,7,10,12,13,15,17,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Random 1000 unique samples saved to: /Users/chrisloukasntais/VsCode/EchoCanvas/dataset.csv


In [7]:
df.head()

,author_name,painting_name,image_url,Genre,Style,Nationality,PaintingSchool,ArtMovement,Field,Date,...,Location,OriginalTitle,Dimensions,Series,Teachers,FriendsandCoworkers,Artinstitution,Period,Theme,Path
0,Gustave Courbet,Woman With A Parrot##AAHozJAL0gqXcA,http://lh3.googleusercontent.com/nPuY2uKye4csy...,NaN,NaN,NaN,NaN,NaN,Painting,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Artists1/Gustave Courbet/Woman With A Parrot##...
1,Auguste Rodin,La Tentation Saint Antoine##WAGC82imJTDyIg,http://lh3.googleusercontent.com/CgGN-xuGOgzBu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Acquis de Rodin Auguste en 1903 avec l'aide d...,NaN,NaN,Artists1/Auguste Rodin/La Tentation Saint Anto...
2,Frida Kahlo,Retrato De Alejandro Gómez Arias##0QFuguLe4xyN_A,http://lh3.googleusercontent.com/j-stBr_Q1mblB...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Artists1/Frida Kahlo/Retrato De Alejandro Góme...
3,Banksy,The Wall Banksy Balloons##FgHoVE-hmt6DBQ,http://lh4.ggpht.com/DajPMSanJJDk7gPghfEqoT5Ru...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,in a settlement in Palestine in the middle east,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Artists1/Banksy/The Wall Banksy Balloons##FgHo...
4,El Greco,The Visitation##HQEQ_qXDtRrzkA,http://lh4.ggpht.com/dYGuyZERVymx98N9Uw3KKvOU6...,NaN,NaN,NaN,NaN,NaN,Painting,ca. 1610-1614,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Artists1/El Greco/The Visitation##HQEQ_qXDtRrz...


In [4]:
import pandas as pd

# Filepath to the CSV file
file_path = "/Users/chrisloukasntais/VsCode/EchoCanvas/full_dataset.csv"

# Load the CSV file
df = pd.read_csv(file_path)

# Count the number of non-null values in each row
df["non_null_count"] = df.notnull().sum(axis=1)

# Find the row with the maximum number of non-null values
row_with_most_values = df.loc[df["non_null_count"].idxmax()]

# Drop the helper column before displaying
row_with_most_values = row_with_most_values.drop("non_null_count")

# Display the row
print("First sample with the most columns having values:")
print(row_with_most_values)

/var/folders/bc/8_g2sm9x4917kl9d15xv74cc0000gn/T/ipykernel_1348/3387213093.py:7: DtypeWarning: Columns (3,5,6,7,10,12,13,15,17,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


First sample with the most columns having values:
author_name                                                Pablo Picasso
painting_name                    Blind Minotaur Is Guided By Girl 1934 2
image_url              https://uploads5.wikiart.org/images/pablo-pica...
Genre                                              mythological painting
Style                                                         Surrealism
Nationality                                                      Spanish
PaintingSchool                                            École de Paris
ArtMovement                                                       Cubism
Field                                         painting,sculpture,drawing
Date                                                                1934
Influencedby           Marc Chagall,Henri Rousseau,El Greco,Francisco...
Media                                                      etching,paper
Influencedon           Marc Chagall,Amedeo Modigliani,Willem de Kooni...
F

In [22]:
import pandas as pd

# Filepath to the CSV file
file_path = "/Users/chrisloukasntais/VsCode/EchoCanvas/full_dataset.csv"

# Load the CSV file
df = pd.read_csv(file_path)

# Define the required columns
required_columns = ["painting_name", "author_name", "Genre", "Style", "Theme", "Period", "Path", "Field", "Date"]

# Filter rows where all required columns have non-null values
df_filtered = df.dropna(subset=required_columns)

# Ensure the "Field" column has the value "painting" for all rows
df_filtered["Field"] = "painting"

# Keep only the required columns
df_filtered = df_filtered[required_columns]

# Check if the filtered dataset has at least 1000 rows
if len(df_filtered) < 1000:
    print(f"Filtered dataset has only {len(df_filtered)} rows. Sampling all available rows.")
    df_sampled = df_filtered  # Use all rows if fewer than 1000
else:
    # Randomly sample 1000 rows
    df_sampled = df_filtered.sample(n=1000, random_state=42)

# Save the sampled data to a new CSV file
output_file_path = "/Users/chrisloukasntais/VsCode/EchoCanvas/final_dataset.csv"
df_sampled.to_csv(output_file_path, index=False)

print(f"Final dataset saved to: {output_file_path}")

/var/folders/bc/8_g2sm9x4917kl9d15xv74cc0000gn/T/ipykernel_1348/1435908448.py:7: DtypeWarning: Columns (3,5,6,7,10,12,13,15,17,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Filtered dataset has only 261 rows. Sampling all available rows.
Final dataset saved to: /Users/chrisloukasntais/VsCode/EchoCanvas/final_dataset.csv


/var/folders/bc/8_g2sm9x4917kl9d15xv74cc0000gn/T/ipykernel_1348/1435908448.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["Field"] = "painting"
